# FASE 7

## Preparación y Entrenamiento de la RNN (LSTM / GRU)

### 1️⃣ Datos
- Dataset final: `../../data/processed/final_features_list.csv.gz`  
- Dimensiones: ~44.630 filas × 90 columnas  
- Rango temporal: **2005-01-01 → 2025-10-31**  
- La serie debe reflejar **exactamente la evolución real de la bolsa**  
- Variables principales: precios y retornos de BBVA y Santander + componentes PCA  



### 2️⃣ Modelado
- Modelos a probar: **LSTM** y **GRU**, comparar desempeño  
- Predicción **corto plazo**: 1 a 5 días adelante  
- Entrada principal: **t-1** (opcional incluir t-2, t-3… para patrones más largos)  
- Considerar **descomposición de la serie** (tendencia / estacionalidad / residual)  

### 3️⃣ Preprocesamiento
- Normalización de datos (z-score o min-max) por ventana de entrenamiento  
- Forward-fill / backward-fill para huecos temporales, sin alterar la curva de la serie  
- Opcional: detrending o trabajar solo con la serie residual  
- Preparar secuencias de entrada (`X`) y salida (`y`) para la RNN  

### 4️⃣ Entrenamiento
- Optimizadores: **Adam** o **descenso de gradiente clásico**  
- **Early stopping** para evitar gradientes explosivos o demasiado pequeños  
- Fijar **semilla** para reproducibilidad  
- **Gradient clipping** para evitar explosión de gradientes  
- Posible uso de **scheduler de learning rate**  
- Monitorear métricas: **logloss, accuracy, RMSE, MAPE**  
- Medir y printear **tiempo por epoch y métricas**  
- Visualización de curvas de pérdida y precisión  

### 5️⃣ Validación
- División temporal: **train / validation / test**  
- Validar predicciones en ventanas cortas y por eventos macroeconómicos  
- Predicción **single-step** vs **multi-step** (iterativa o directa)  
- Graficar predicciones vs real, especialmente en picos, crisis y shocks  

### 6️⃣ Experimentación
- Iterar **hiperparámetros**: tamaño de la RNN, número de capas, dropout, learning rate, batch size  
- Decidir si usar **GPU** para acelerar entrenamiento  



### 7️⃣ Notas adicionales
- Mantener consistencia temporal en la normalización y splits  
- Todas las decisiones de imputación o relleno deben preservar la integridad de la curva de mercado  
- Preparar el set final de features (`top_features + PCA components`) listo para RNN  

---


## Plan de Notebook 



### **Parte 1 — Configuración Inicial y Carga de Datos**

**Objetivos:**
- Cargar el dataset final (`final_features_list.csv.gz`)  
- Filtrar por rango temporal 2005–2025  
- Fijar semilla para reproducibilidad  
- Seleccionar columnas principales (BBVA, Santander y PCA)  

**Consideraciones:**
- Mantener la curva de la bolsa **exactamente igual**  
- Dataset ~44.630 × 90 columnas  
- Forward/backward fill solo si hay huecos, sin alterar la serie  



### **Parte 2 — Preprocesamiento y Normalización**

**Objetivos:**
- Escalar variables (z-score o min-max por ventana de entrenamiento)  
- Forward-fill / backward-fill de valores faltantes  
- Opcional: detrending o trabajar con la serie residual  
- Crear features de lag: t-1 (y opcional t-2, t-3…)  
- Preparar `X` y `y` para predicción 1–5 días adelante  
- Guardar set listo para train/validation/test  

**Notas importantes:**
- Mantener integridad temporal  
- No rellenar con medias globales si hay eventos significativos  

### **Parte 3 — División Temporal y Secuencias**

**Objetivos:**
- Dividir en train / validation / test por fechas  
- Crear secuencias deslizantes (`t-1 → t-n`) para la RNN  
- Manejar batchs y padding si hay huecos  
- Preparar secuencias para multi-step prediction (1–5 días)  

**Notas:**
- Validación estrictamente temporal (no shuffle)  
- Mantener consistencia con la curva real de mercado  

### **Parte 4 — Definición del Modelo**

**Objetivos:**
- Definir **LSTM y GRU**  
- Ajustar hiperparámetros: capas, unidades, dropout  
- Optimizador: Adam o SGD  
- Loss function: MSE o RMSE  
- Gradient clipping y early stopping  
- Configurar GPU si disponible  

**Notas:**
- Monitorear logloss, accuracy, RMSE, MAPE  
- Guardar la arquitectura y pesos para comparaciones  

### **Parte 5 — Entrenamiento**

**Objetivos:**
- Entrenar modelo por epochs  
- Mostrar **tiempo por epoch**  
- Imprimir métricas de cada epoch  
- Early stopping basado en validation loss  
- Usar scheduler de learning rate si se estanca  

**Notas:**
- Guardar curvas de loss y accuracy  
- Guardar modelo final entrenado  

### **Parte 6 — Validación y Evaluación**

**Objetivos:**
- Graficar predicciones vs real  
- Comparar single-step vs multi-step  
- Analizar picos, crisis y shocks macroeconómicos  
- Calcular métricas financieras: RMSE, MAPE, Sharpe ratio simulado  

**Notas:**
- Evaluar desempeño por ventanas de 1–5 días  
- Revisar que la predicción siga la curva de mercado exactamente  

### **Parte 7 — Experimentación y Hiperparámetros**

**Objetivos:**
- Iterar sobre unidades, capas, dropout, learning rate, batch size  
- Comparar LSTM vs GRU  
- Analizar resultados por cada combinación de hiperparámetros  
- Preparar conclusiones sobre la mejor configuración  

**Notas:**
- Registrar todos los experimentos y métricas  
- Decidir si usar GPU para acelerar experimentación  


## Código

In [10]:
# ==============================
# Liberries
# ==============================
import pandas as pd
import numpy as np
import torch
import random
from IPython.display import display

In [15]:
# ==============================
# PARTE 1 — Configuración Inicial y Carga de Datos
# ==============================

# ------------------------------
# 1. Fijar semilla para reproducibilidad
# ------------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ------------------------------
# 2. Cargar dataset final y revisar columnas
# ------------------------------
data_path = "../../data/processed/final_features_list.csv.csv.gz"
data = pd.read_csv(data_path, compression='gzip')

print("Columnas del dataset:", list(data.columns))

# Asegurarnos de que Date se reconoce como datetime
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
else:
    raise ValueError("No se encuentra la columna 'Date'. Revisa tu CSV")

print(f"Dimensiones dataset original: {data.shape}")

# ------------------------------
# 3. Filtrar rango temporal
# ------------------------------
start_date = "2005-01-01"
end_date = "2025-10-31"
data = data.loc[start_date:end_date]

print(f"Dimensiones dataset filtrado 2005-2025: {data.shape}")

# ------------------------------
# 4. Seleccionar columnas principales
# ------------------------------
# Columnas de interés para la RNN: precios BBVA, Santander y PCA
pca_cols = [col for col in data.columns if "PCA" in col]
main_cols = ['BBVA.MC_Close', 'SAN.MC_Close'] + pca_cols

data_rnn = data[main_cols]
print(f"Columnas seleccionadas para la RNN: {data_rnn.shape[1]}")

# ------------------------------
# 5. Visualización rápida
# ------------------------------
display(data_rnn.head())

FileNotFoundError: [Errno 2] No such file or directory: '../../data/processed/final_features_list.csv.csv.gz'